## Visualizing your graph

- The easiest way to create a visualization is to pass a `tf.Graph` when creating the `tf.summary.FileWriter`:

```python
# Build your graph.
x = tf.constant([[37.0, -23.0], [1.0, 4.0]])
w = tf.Variable(tf.random_uniform([2, 2]))
y = tf.matmul(x, w)
# ...
loss = ...
train_op = tf.train.AdagradOptimizer(0.01).minimize(loss)

with tf.Session() as sess:
  # `sess.graph` provides access to the graph used in a `tf.Session`.
  writer = tf.summary.FileWriter("/tmp/log/...", sess.graph)

  # Perform your computation...
  for i in range(1000):
    sess.run(train_op)
    # ...

  writer.close()
```


![](https://www.tensorflow.org/images/mnist_deep.png)

- When training a model, a common way of organizing your code is to use one graph for training your model, and **a separate graph for evaluating or performing inference with a trained model**. In many cases, the inference graph will be different from the training graph: for example, techniques like dropout and batch normalization use different operations in each case.

- by default utilities like `tf.train.Saver` use the names of tf.Variable objects (which have names based on an underlying tf.Operation) to identify each variable in a saved checkpoint. When programming this way, you can either use completely separate Python processes to build and execute the graphs, or you can use multiple graphs in the same process.
- TensorFlow provides a "default graph" that is implicitly passed to all API functions in the same context.
- A tf.Graph defines the namespace for tf.Operation objects: each operation in a single graph must have a unique name. TensorFlow will "uniquify" the names of operations by appending "_1", "_2", and so on to their names if the requested name is already taken. Using multiple explicitly created graphs gives you more control over what name is given to each operation.
- The default graph stores information about every tf.Operation and tf.Tensor that was ever added to it. If your program creates a large number of unconnected subgraphs, it may be more efficient to use a different tf.Graph to build each subgraph, so that unrelated state can be garbage collected.

In [3]:
import tensorflow as tf
g_1 = tf.Graph()
with g_1.as_default():
  # Operations created in this scope will be added to `g_1`.
  c = tf.constant("Node in g_1")

  # Sessions created in this scope will run operations from `g_1`.
  sess_1 = tf.Session()

g_2 = tf.Graph()
with g_2.as_default():
  # Operations created in this scope will be added to `g_2`.
  d = tf.constant("Node in g_2")

# Alternatively, you can pass a graph when constructing a `tf.Session`:
# `sess_2` will run operations from `g_2`.
sess_2 = tf.Session(graph=g_2)

assert c.graph is g_1
assert sess_1.graph is g_1

assert d.graph is g_2
assert sess_2.graph is g_2

In [11]:
# Print all of the operations in the default graph.
g = tf.get_default_graph()
g.get_operations()

[]

In [7]:
g_1.get_operations()

[<tf.Operation 'Const' type=Const>]

In [12]:
g_2.get_operations()

[<tf.Operation 'Const' type=Const>]

In [14]:
g.get_operations?

In [15]:
g.get_collection?